In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import math
import numpy as np
from scipy.stats import norm

In [3]:
from keras.layers import Input, Dense, Activation, Lambda, Embedding, Reshape, RepeatVector
from keras.layers import merge
from keras.models import Model
from keras import regularizers
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras import objectives

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled)


### References :
##### VAE with GMM : https://arxiv.org/pdf/1611.05148.pdf
##### Categorical VAE parameterization : http://blog.evjang.com/2016/11/tutorial-categorical-variational.html
##### Auto-Encoding Variational Bayes : https://arxiv.org/abs/1312.6114
##### Tutorial from Oliver Durr : https://home.zhaw.ch/~dueo/bbs/files/vae.pdf
##### Building autoencoders in keras : https://blog.keras.io/building-autoencoders-in-keras.html

## KL between 2 normal distributions

* a) if N(0,1)^d if one of the 2 distributions

       KL = 0.5 SUM_OVER_DIMS[ -1 - log_var + square(mean) + var ]

* b) for 2 diagonal covariances matrices

       KL = 0.5 SUM_OVER_DIMS[ -1 + log(var2)-log(var1) + var1/var2 + (mean1-mean2)^2 / var2 ]
       
* c) see most general formula at :

       stats.stackexchange.com/questions/60680/kl-divergence-between-two-multivariate-gaussians/60699#60699

In [4]:
from keras.datasets import mnist
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

## Variational Auto-Encoder with Gaussian Mixture generator

In [5]:
batch_size = 16
original_dim = 784
latent_dim = 2

NM = 10 # number of mixture clusters

intermediate_dim1 = 500
intermediate_dim2 = 500

nb_epoch = 25
epsilon_std = 1.0

#### Reparameterization trick

In [6]:
# Approximate Multinomial distrib. with a Gumbel-Softmax distrib.
# see documentation at :
# http://blog.evjang.com/2016/11/tutorial-categorical-variational.html
def sampling_categorical(c_logits):
    temperature = 0.1 # the lower it is the sharper the function is
    eps = 1e-20
    U = K.random_uniform(shape=(NM,), low=0, high=1)
    gumbel_noise = -K.log(-K.log(U + eps) + eps)
    # WARNING : dont return softmax as we apply it NEXT
    return (c_logits + gumbel_noise) / temperature
    #return K.softmax((c_logits+gumbel_noise)/temperature)

def sampling_normal(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              std=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

#### Lambda functions

In [7]:
def log_pdf_normal(args):
    # WARNING : assumes covariance is diagonal
    # INFO : returns log of pdf for softmax function
    mu, log_var, z = args
    return -0.5 * ( K.sum((mu - z) * (mu - z) / K.exp(log_var))
                    + K.sum(log_var)
                    + latent_dim * math.log(2*math.pi) )

def kl_normal(args):
    i_mean, i_log_var, z_mean, z_log_var = args
    return -0.5 * K.sum( 1 
                        - i_log_var + z_log_var
                        - K.exp(z_log_var)/K.exp(i_log_var)
                        - K.square(z_mean - i_mean) / K.exp(i_log_var) , axis=-1)

In [23]:
x = Input(batch_shape=(batch_size, original_dim))
h1 = Dense(intermediate_dim1, activation='softplus')(x)
h2 = Dense(intermediate_dim2, activation='softplus')(h1)
z_mean = Dense(latent_dim, activation=None)(h2)
z_log_var = Dense(latent_dim, activation=None)(h2)

#ONEINT = Input(batch_shape=(batch_size, 1))

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling_normal, output_shape=(latent_dim,))([z_mean, z_log_var])


E_mean = Embedding(NM, latent_dim, input_length=1) # WARNING : needs near-0 initialization
E_log_var = Embedding(NM, latent_dim, input_length=1) # WARNING : needs near-1 initialization

logpdfs = []
kl_losses = []
for i in range(NM):
    # Tensor with constant i value
    dummy_d = Dense(1, weights=[np.zeros((original_dim, 1)),i*np.ones((1,))], input_dim=original_dim)
    dummy_d.trainable = False
    ohe_i = dummy_d(x)
    # Apply tensor to Mean and Var embeddings
    i_mean = Reshape((latent_dim,))( E_mean(ohe_i) )
    i_log_var = Reshape((latent_dim,))( E_log_var(ohe_i) )
    # PDF
    logpdfs.append( Reshape((1,))( Lambda(log_pdf_normal, output_shape=(1,))([i_mean, i_log_var, z]) ) )
    # Compute KL-loss terms
    kl_losses.append( Reshape((1,))( Lambda(kl_normal, output_shape=(1,))([i_mean, i_log_var, z_mean, z_log_var]) ) )


#logpdfmat = Reshape((NM,1))( merge(logpdfs, mode='concat', concat_axis=1) ) # .summary to check axis
logpdfmat = merge(logpdfs, mode='concat', concat_axis=1) # .summary to check axis
c_logits = Activation('softmax')(logpdfmat)
# WARNING : need to compute c_logits from z
c = Activation('softmax')( Lambda(sampling_categorical, output_shape=(NM,))(c_logits) )

klmat = Reshape((NM,))( merge(kl_losses, mode='concat', concat_axis=1) )

kl_loss = merge([c, klmat], mode='dot') # WARNING : needs to be-log(SUM(prob_j * exp(-KL_j)))

##############################

#inputs = Input(shape=(nn_input_dim,))
inputs = x
#outputs = Input(shape=(8,)) # in AE model output=input=x
outputs = x

predictions = []
for i in range(NM):
    decoder_h1 = Dense(intermediate_dim1, activation='softplus')
    decoder_h2 = Dense(intermediate_dim2, activation='softplus')
    decoder_proba = Dense(original_dim, activation='sigmoid')
    h1_decoded = decoder_h1(z)
    h2_decoded = decoder_h2(h1_decoded)
    i_x_decoded_proba = decoder_proba(h2_decoded) # a Bernoulli dist. has a single prob. parameter
    predictions.append( i_x_decoded_proba )


x_decoded_proba = predictions[0]

'''
predmat = Reshape((NM,original_dim))( merge(predictions, mode='concat', concat_axis=1) ) #.summary to check axis

deltas = merge([RepeatVector(NM)(outputs), predmat], output_shape=(NM,original_dim), mode=lambda x: -(x[0] * K.log(x[1])))

deltasums = Lambda(lambda x: K.sum(x, axis=2), output_shape=lambda s: (s[0], s[1]))(deltas)# .summary to check axis

hinton_trick = True # see "Adaptive Mixtures of Local Experts"
if hinton_trick:
    Hinton1 = Lambda(lambda x: K.exp(-x), output_shape=lambda s: s)
    deltasums = Hinton1(deltasums)

# WARNING : gate is just the sampled c vector
gate = c

errors = merge([gate, deltasums], mode='dot')
##############################
'''

'\npredmat = Reshape((NM,original_dim))( merge(predictions, mode=\'concat\', concat_axis=1) ) #.summary to check axis\n\ndeltas = merge([RepeatVector(NM)(outputs), predmat], output_shape=(NM,original_dim), mode=lambda x: -(x[0] * K.log(x[1])))\n\ndeltasums = Lambda(lambda x: K.sum(x, axis=2), output_shape=lambda s: (s[0], s[1]))(deltas)# .summary to check axis\n\nhinton_trick = True # see "Adaptive Mixtures of Local Experts"\nif hinton_trick:\n    Hinton1 = Lambda(lambda x: K.exp(-x), output_shape=lambda s: s)\n    deltasums = Hinton1(deltasums)\n\n# WARNING : gate is just the sampled c vector\ngate = c\n\nerrors = merge([gate, deltasums], mode=\'dot\')\n##############################\n'

In [31]:
def vae_loss(x, x_decoded_proba):
    xent_loss = original_dim * objectives.binary_crossentropy(x, x_decoded_proba)
    #kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss# + kl_loss

vae = Model(x, x_decoded_proba)
vae.compile(optimizer=Adam(1e-3), loss=vae_loss)

In [32]:
#mdl = Model([x, ONEINT], [logpdfs[0], logpdfs[1], logpdfs[2], logpdfs[3], logpdfs[4]])
#mdl = Model([x, ONEINT], logpdfmat)
#mdl = Model([x, ONEINT], c_logits)
#mdl = Model(x, c)
#mdl = Model(x, kl_losses[0])
#mdl = Model(x, klmat)
mdl = Model(x, kl_loss)
#mdl = Model([x, ONEINT], predmat)
#mdl = Model([x, ONEINT], deltas)
#mdl = Model([x, ONEINT], deltasums)
#mdl.summary()

In [37]:
#onearr = np.ones((batch_size, 1)).astype(int)
print mdl.predict(x_train[0:batch_size,:]).shape
np.array(mdl.predict(x_train[0:batch_size,:]))[0:3,:]

(16, 1)


array([[ 10.58854771],
       [ 10.58854771],
       [ 10.58854771]], dtype=float32)

In [36]:
print vae.predict(x_train[0:batch_size,:]).shape
np.array(vae.predict(x_train[0:batch_size,:]))[0:3,:]

(16, 784)


array([[  3.01692512e-06,   2.89611307e-06,   5.49303468e-06, ...,
          2.95742734e-06,   1.25978045e-06,   1.98960765e-06],
       [  7.53871234e-07,   2.33677406e-06,   8.20605965e-06, ...,
          9.93965955e-07,   4.22242283e-06,   2.02209208e-06],
       [  1.51504019e-05,   1.18276748e-05,   3.19873943e-05, ...,
          3.02769040e-05,   2.15850432e-05,   3.32489908e-05]], dtype=float32)

In [35]:
vae.fit(x_train, x_train,
        shuffle=True,
        nb_epoch=1, # may have to increase this value
        batch_size=batch_size)
        #validation_data=(x_test, x_test))

Epoch 1/1
60000/60000 [==============================] - 26s - loss: 156.9996    


In [ ]:
# build a model to project inputs on the latent space
encoder_mean = Model(x, z_mean)
#encoder_stdev = Model(x, K.exp(z_log_var / 2))

#encoder = Model(x, z)

# build a digit generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h1_decoded = decoder_h1(decoder_input)
_h2_decoded = decoder_h2(_h1_decoded)
_x_decoded_proba = decoder_proba(_h2_decoded)
generator = Model(decoder_input, _x_decoded_proba)